# 0. 코랩 이용 준비
---

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ls /content/gdrive/MyDrive/Deep_Learning/

'cbow 실험.ipynb'				 Transfer_Learning
 colab_사용법.ipynb				 Untitled12.ipynb
 dataset					'word2vec 오류난 부분.ipynb'
'모델을 구글 드라이브에 저장하기 실험실.ipynb'	 word2vec_수정.ipynb
 Learned_Parameters				 word2vec.ipynb


In [4]:
%cd /content/gdrive/MyDrive/Deep_Learning/

/content/gdrive/MyDrive/Deep_Learning


In [5]:
!pwd

/content/gdrive/MyDrive/Deep_Learning


# 1. 라이브러리 호출
---

In [6]:
import torch
import torch.nn as nn
import collections # collections 모듈에는 데이터 처리를 위한 유용한 객체가 많이 있다. https://wikidocs.net/84392
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import time

from dataset import ptb

# 2. 사용할 함수 추가
---

In [7]:
def create_contexts_target(corpus, window_size=1):
    '''맥락과 타깃 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return:
    '''
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

# 3. device 설정
---

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #  torch.cuda.is_available() GPU를 사용가능하면 True, 아니라면 False를 리턴

print("지금 사용하는 device :",device)

지금 사용하는 device : cuda:0


In [9]:
print(torch.cuda.is_available())

True


# 4. 데이터셋 준비
---

In [10]:
corpus, word_to_id, id_to_word = ptb.load_data('train')

In [11]:
contexts, target = create_contexts_target(corpus, 1)

임베딩에 들어가야 할 것은 long나 int인데

파이토치로 역전파 계산하려면 float와 requires_grad=True가 필요한데 이 부분은 다음 참고

https://hongl.tistory.com/244

In [12]:
class Corpus_Dataset(Dataset):
    def __init__(self, corpus, window_size = 1):
        contexts, target = create_contexts_target(corpus, window_size)
        self.contexts = contexts
        self.target = target

    def __len__(self):
        return len(self.contexts)

    def __getitem__(self, idx):
        contexts = torch.tensor(self.contexts[idx], dtype = torch.long)
        target = torch.tensor(self.target[idx], dtype = torch.long)
        return contexts, target

데이터셋 준비 완료

In [13]:
dataset = Corpus_Dataset(corpus, window_size = 2)
dataloader = DataLoader(dataset, batch_size = 100)

# 5. 신경망 생성
---

### 5.1 다음은 오류가 난 모델의 일부분이다. 

In [14]:
class pre_h(nn.Module):
    def __init__(self, vocab_size, hidden_size, window_size):
        super().__init__()
        self.window_size = window_size
        self.in_layer = nn.Embedding(vocab_size, hidden_size)
        self.weight_list = []

    def forward(self, contexts):
        for i in range(2 * self.window_size):
            weight = self.in_layer(contexts[:,i])
            self.weight_list.append(weight)

        h = sum(self.weight_list)
        h = h / len(self.weight_list)

        return h


### 5.2 다음은 오류를 수정한 모델의 일부분이다.

In [15]:
class pre_h(nn.Module):
    def __init__(self, vocab_size, hidden_size, window_size):
        super().__init__()
        self.window_size = window_size
        self.in_layer = nn.Embedding(vocab_size, hidden_size)
        # self.weight_list = []

    def forward(self, contexts):
        # 우민혁: h의 결과들을 리스트에 저장해서 sum하는 부분에서 에러가 발생한듯
        # 구체적인 이유는 모르겠음.
        # 따라서 에러를 방지하기 위해 반복문에 돌 때마다 바로 h = h + weight을 함
        h = 0
        for i in range(2 * self.window_size):
            weight = self.in_layer(contexts[:,i])
            # self.weight_list.append(weight)
            h = h + weight
        h = h / int(2 * self.window_size)
        # h = sum(self.weight_list)
        # h = h / len(self.weight_list)

        return h


왜 오류가 난건지 정확히는 불명.

그 이유도 구글링해봐도 나와있진 않는다. 

그저 겉면은 파이썬이나 속은 C++로 돌아가는 파이토치의 특성상 파이썬 기본 라이브러리가 잘 먹히지 않는듯...?


In [16]:
vocab_size = len(id_to_word)
print(vocab_size)

10000


In [18]:
class UnigramSampler:
    '''
    말뭉치의 통계 데이터를 기초로 샘플링 하는 클래스
    말뭉치에서 각 단어의 출현 횟수를 구해 '확률분포'로 나타낸 다음 확률 분포대로 단어를 샘플링함
    '''
    def __init__(self, corpus, power, sample_size, device):
        super().__init__()
        self.sample_size = sample_size # 샘플링할 갯수
        self.vocab_size = None
        self.word_p = None
        self.device = device

        counts = collections.Counter()

        for word_id in corpus: # 말뭉치에서 단어를 꺼낸다 (이때 말뭉치는 단어 id로 이루어진 행렬이다.)
            counts[word_id] += 1 # 특정 단어가 몇번 나왔는지 숫자를 센다.

        vocab_size = len(counts) # 총 단어의 갯수 이는 Counter의 특징을 알아야 한다.
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size) # 단어의 갯수 만큼 행렬 생성

        for i in range(vocab_size):
            self.word_p[i] = counts[i] # 각 단어가 몇번 나왔었는지 텐서에 저장

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p) # 기본 확률 분포에 0.75 제곱

    def get_negative_sample(self, target): # 네거티브 샘플링
        batch_size = target.shape[0]

        negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size), replace=True, p=self.word_p)
        negative_sample = torch.tensor(negative_sample).to(self.device) # 여기에 to(device)를 안하면 비교 대상이 GPU에 안올라가서 오류남

        return negative_sample


In [19]:
class NegativeSampling(nn.Module):
    def __init__(self, vocab_size, hidden_size, corpus, device, power = 0.75, sample_size = 5):
        super().__init__()
        self.sample_size = sample_size # 네거티브 샘플링 할 횟수
        self.sampler = UnigramSampler(corpus, power, sample_size, device) # 확률 분포에 따라서 샘플링 지정

        self.embedding = nn.Embedding(vocab_size, hidden_size) # (hidden_size, vocab_size)
        self.sigmoid = nn.Sigmoid()
        self.loss_layer = nn.CrossEntropyLoss()

        self.hidden_size = hidden_size

    def forward(self, h, target):
        loss_data = 0

        target = target.detach()

        # 긍정적 예 샘플링
        w = self.embedding(target)
        w = torch.squeeze(w).reshape(self.hidden_size, -1)
        loss = h @ w
        loss = self.sigmoid(loss)
        correct_label = torch.ones_like(loss)
        loss = self.loss_layer(loss, correct_label)
        loss_data += loss

        # 부정적 예로 타겟 변경
        negative_target = self.sampler.get_negative_sample(target)

        # 부정적 예 샘플링

        for i in range(self.sample_size):
            w = self.embedding(negative_target[:,i])
            w = torch.squeeze(w).reshape(self.hidden_size, -1)
            loss = h @ w
            loss = self.sigmoid(loss)

            negative_label = torch.zeros_like(loss)
            loss = self.loss_layer(loss, negative_label)

            loss_data += loss

        return loss_data

In [18]:
class sample_cbow(nn.Module): # 그저 단순히 두 부품을 합쳐봄
    def __init__(self, vocab_size, hidden_size, corpus, window_size, power = 0.75, sample_size = 5):
        super().__init__()

        self.model_1 = pre_h(vocab_size, hidden_size, window_size)
        self.model_2 = NegativeSampling(vocab_size, hidden_size, corpus, power, sample_size)

    def forward(self, contexts, target):
        out = self.model_1(contexts)
        out = self.model_2(out, target)

        return out



In [23]:
vocab_size = len(id_to_word)
model = sample_cbow(vocab_size, 100, corpus, 1)

learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 아담으로 옵티마이저 설정

i = 1


for contexts, target in dataloader:
    print('===================================')
    print(i,'번째 실험')

    model.to(device) 
    contexts.to(device)
    target.to(device)

    output = model(contexts, target)
    optimizer.zero_grad()
    output.backward(retain_graph=True)
    optimizer.step()
   
    i += 1



1 번째 실험
2 번째 실험
3 번째 실험
4 번째 실험
5 번째 실험
6 번째 실험
7 번째 실험
8 번째 실험
9 번째 실험
10 번째 실험
11 번째 실험
12 번째 실험
13 번째 실험
14 번째 실험
15 번째 실험
16 번째 실험
17 번째 실험
18 번째 실험
19 번째 실험
20 번째 실험
21 번째 실험
22 번째 실험
23 번째 실험
24 번째 실험
25 번째 실험
26 번째 실험
27 번째 실험
28 번째 실험
29 번째 실험
30 번째 실험
31 번째 실험
32 번째 실험
33 번째 실험
34 번째 실험
35 번째 실험
36 번째 실험
37 번째 실험
38 번째 실험
39 번째 실험
40 번째 실험
41 번째 실험
42 번째 실험
43 번째 실험
44 번째 실험
45 번째 실험
46 번째 실험
47 번째 실험
48 번째 실험
49 번째 실험
50 번째 실험
51 번째 실험
52 번째 실험
53 번째 실험
54 번째 실험
55 번째 실험
56 번째 실험
57 번째 실험
58 번째 실험
59 번째 실험
60 번째 실험
61 번째 실험
62 번째 실험
63 번째 실험
64 번째 실험
65 번째 실험
66 번째 실험
67 번째 실험
68 번째 실험
69 번째 실험
70 번째 실험
71 번째 실험
72 번째 실험
73 번째 실험
74 번째 실험
75 번째 실험
76 번째 실험
77 번째 실험
78 번째 실험
79 번째 실험
80 번째 실험
81 번째 실험
82 번째 실험
83 번째 실험
84 번째 실험
85 번째 실험
86 번째 실험
87 번째 실험
88 번째 실험
89 번째 실험
90 번째 실험
91 번째 실험
92 번째 실험
93 번째 실험
94 번째 실험
95 번째 실험
96 번째 실험
97 번째 실험
98 번째 실험
99 번째 실험
100 번째 실험
101 번째 실험
102 번째 실험
103 번째 실험
104 번째 실험
105 번째 실험
106 번째 실험
107 번째 실험
108 번째 실험
109 번째 실험
110 번째 실험
111 번째 실

KeyboardInterrupt: ignored

# 6. 모델 설정 완료
---

### word2vec 모델 최종 완성
pre_h와 NegativeSampling을 합치자

In [20]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, hidden_size, corpus, window_size, power = 0.75, sample_size = 5):
        super().__init__()
        self.window_size = window_size
        self.in_layer = nn.Embedding(vocab_size, hidden_size)
        self.out_layer = NegativeSampling(vocab_size, hidden_size, corpus, device, power, sample_size)

    def forward(self, contexts, target):
        h = 0
        for i in range(2 * self.window_size):
            weight = self.in_layer(contexts[:,i])
            h = h + weight
        h = h / int(2 * self.window_size)

        out = self.out_layer(h, target)

        return out

    def get_word_vecs(self):
        self.w_in = self.in_layer.weight.data
        return self.w_in

### 모델 학습

In [21]:
def train_model(model, dataloaders, optimizer, device, num_epochs=5):
    since = time.time()
    loss_history = [] # 각 에폭마다 손실함수값 저장
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        running_loss = 0.0 # 손실 함수
        running_corrects = 0 # 정답갯수

        for contexts, targets in dataloaders:
            contexts = contexts.to(device)
            targets = targets.to(device)

            outputs = model(contexts, targets) 
            optimizer.zero_grad()
            outputs.backward() 
            optimizer.step() 

            running_loss += outputs.item() 

        epoch_loss = running_loss / len(dataloaders.dataset)

        print('Loss: {:.4f} '.format(epoch_loss)) 
        loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since # 실행 시간 계산
    print(f'실행 시간 : {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return loss_history


In [22]:
# 하이퍼 파라미터 설정
vocab_size = len(id_to_word)
hidden_size = 100
window_size = 5
learning_rate = 0.001

# device 위치 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("지금 사용하는 device :",device)

# 데이터셋 준비
corpus, word_to_id, id_to_word = ptb.load_data('train')

contexts, target = create_contexts_target(corpus, window_size)

dataset = Corpus_Dataset(corpus, window_size)

dataloader = DataLoader(dataset, batch_size = 100)

# 모델 설정
model = CBOW(vocab_size, hidden_size, corpus, window_size)
model.to(device)

# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 아담으로 옵티마이저 설정


지금 사용하는 device : cuda:0


In [23]:
# 모델 학습
loss_history = train_model(model, dataloader, optimizer, device, num_epochs = 100)

Epoch 0/99
----------
Loss: 4.6344 

Epoch 1/99
----------
Loss: 4.6087 

Epoch 2/99
----------
Loss: 4.6052 

Epoch 3/99
----------
Loss: 4.6052 

Epoch 4/99
----------
Loss: 4.6052 

Epoch 5/99
----------
Loss: 4.6052 

Epoch 6/99
----------
Loss: 4.6052 

Epoch 7/99
----------
Loss: 4.6052 

Epoch 8/99
----------
Loss: 4.6052 

Epoch 9/99
----------
Loss: 4.6052 

Epoch 10/99
----------
Loss: 4.6052 

Epoch 11/99
----------
Loss: 4.6052 

Epoch 12/99
----------
Loss: 4.6052 

Epoch 13/99
----------
Loss: 4.6052 

Epoch 14/99
----------
Loss: 4.6052 

Epoch 15/99
----------
Loss: 4.6052 

Epoch 16/99
----------
Loss: 4.6052 

Epoch 17/99
----------
Loss: 4.6052 

Epoch 18/99
----------
Loss: 4.6052 

Epoch 19/99
----------
Loss: 4.6052 

Epoch 20/99
----------
Loss: 4.6052 

Epoch 21/99
----------
Loss: 4.6052 

Epoch 22/99
----------
Loss: 4.6052 

Epoch 23/99
----------
Loss: 4.6052 

Epoch 24/99
----------
Loss: 4.6052 

Epoch 25/99
----------
Loss: 4.6052 

Epoch 26/99
----------

이제 나온 가중치를 저장해주자

In [24]:
w_in = model.get_word_vecs()
print(w_in)

tensor([[-0.9449, -0.5849,  0.0161,  ..., -1.0903, -0.9557, -0.5309],
        [-0.1289, -0.0103,  1.3929,  ..., -1.5256,  0.0590,  0.1824],
        [-1.3700,  0.3019, -0.8534,  ...,  0.6650, -1.5444, -1.7060],
        ...,
        [ 1.4783,  1.9061, -0.9655,  ..., -1.3951,  0.0530, -0.5620],
        [-0.3938,  2.5629,  1.2125,  ...,  1.4927,  1.9931, -0.1719],
        [ 1.7126, -0.2137, -0.9912,  ...,  0.5755,  2.7250,  0.1613]],
       device='cuda:0')


In [25]:
torch.save(w_in,'./word_vecs.t')

npy로도 저장해두겠다.

In [28]:
w_in = w_in.cpu().numpy()

AttributeError: ignored

In [29]:
print(w_in)

[[-0.94492143 -0.5848655   0.01614317 ... -1.0903388  -0.9557403
  -0.5308652 ]
 [-0.1288693  -0.01029203  1.3928933  ... -1.5256437   0.05897268
   0.1823535 ]
 [-1.3700342   0.30185702 -0.8534287  ...  0.66502535 -1.5444052
  -1.7059944 ]
 ...
 [ 1.4783314   1.9061414  -0.965526   ... -1.3950768   0.05301956
  -0.5619554 ]
 [-0.39383167  2.5629487   1.2125373  ...  1.4927211   1.9930971
  -0.17194422]
 [ 1.7126251  -0.21373655 -0.99115103 ...  0.5755051   2.7250323
   0.16133635]]


In [30]:
np.save('./npy_word_vecs',w_in)